# Regresji grzbietowa i Elastic-Net, alfa = lambda

# Import bibliotek

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import ElasticNet

# Import danych

In [2]:
data_read = np.loadtxt("http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/australian/australian.dat")
df = pd.DataFrame(data_read)
data = df.rename(columns = {14: 'class'})

# Funkcja normalizacji danych

In [ ]:
def scale(df): 
    data_scale = df.copy()
    for column in data_scale.columns:
        data_scale[column] = (data_scale[column] - data_scale[column].mean()) / data_scale[column].std()
        return data_scale

# Podzial danych / normalizacja

In [3]:
x = data.drop(columns=['class'])
y = pd.DataFrame(data['class'])

#Normalizacja
xscale = scale(x)

#Losowy podzial danych na zbiór treningowy (80%) i testowy (20%) 
xtrain, xtest, ytrain, ytest = train_test_split(xscale,y, test_size = .20)

# Regresja grzbietowa

In [4]:
#Funkcja uczaca model dla zmiennego poziomu alfa

def reg_grzbietowa(alfa):
    ridge_reg = Ridge(alpha = alfa)
    ridge_reg.fit(xtrain, ytrain)
    ypred = ridge_reg.predict(xtest)
    ypred_df = pd.DataFrame(ypred,columns=['ypred'])
    return ypred_df

In [5]:
#Obliczanie trafnosci dla alpha = 1
tn, fp, fn, tp = confusion_matrix(ytest,reg_grzbietowa(1) > 0.5).ravel()

accuracy =((tp + tn)/(tn + fp + fn + tp))
print ('Trafnosc dla alpha 1:', accuracy)

Trafnosc dla alpha 1: 0.8695652173913043


In [6]:
#Funkcja wyliczajaca optymalna wartosc alpha
alpha_values = np.concatenate((0.00001,0.0001,0.001,np.arange(0.01, 1.01, 0.01), np.arange(0, 1000, 10)),axis=None)

wyniki = {}
for punkt in alpha_values:
    tn2, fp2, fn2, tp2 = confusion_matrix(ytest,reg_grzbietowa(punkt) > 0.5).ravel()
    accuracy=((tp2 + tn2)/(tn2 + fp2 + fn2 + tp2))
    wyniki[punkt] = accuracy

wyniki = pd.Series(wyniki)
print(wyniki)
najwyzsza_trafnosc = wyniki.iloc[wyniki.argmax()]
najlepsza_alpha = wyniki.iloc[wyniki.argmax():wyniki.argmax() + 1].index[0]
print('Najwyzsza trafnosc:',najwyzsza_trafnosc)
print('Alpha dal tej trafnosci:',najlepsza_alpha)

0.00001      0.869565
0.00010      0.869565
0.00100      0.869565
0.01000      0.869565
0.02000      0.869565
               ...   
950.00000    0.775362
960.00000    0.775362
970.00000    0.775362
980.00000    0.775362
990.00000    0.775362
Length: 203, dtype: float64
Najwyzsza trafnosc: 0.8768115942028986
Alpha dal tej trafnosci: 60.0


# Elastic Net

In [7]:
#Funkcja uczaca model dla zmiennego poziomu alfa oraz l1 ratio

def elastic_net(alfa, l1ratio):
    elastic_net = ElasticNet(alpha = alfa, l1_ratio = l1ratio)
    elastic_net.fit(xtrain, ytrain)
    ypred_elastic = elastic_net.predict(xtest)
    ypred_elastic_df = pd.DataFrame(ypred_elastic,columns = ['ypred'])
    return ypred_elastic_df

In [8]:
#Obliczanie trafnosci dla alpha = 1 / l1 ratio = 0,5
tn2, fp2, fn2, tp2 = confusion_matrix(ytest,elastic_net(1,0.5) > 0.5).ravel()

accuracy_elastic=((tp2 + tn2)/(tn2 + fp2 + fn2 + tp2))
print ('Trafnosc dla alpha 1 / l1 ratio 0.5:', accuracy_elastic)

Trafnosc dla alpha 1 / l1 ratio 0.5: 0.7318840579710145


# Funkcja wyliczajaca trafnosc dla roznych kombinacji alpha i l1 ratio.  
Oczywiscie mozna powiekszyc przeszukiwane zbiory alpha oraz l1 ratio aby znalezc dokladniejszy wynik.  
Na potrzeby zadania zdecydowalem sie jednak ograniczyc liczbe potencjalnych odpowiedzi w celu przyspieszenia  
dzialania funkcji

In [10]:
l1_ratio_values = np.arange(0.01, 1, 0.01)
alpha_values_elastic = [0.00001,0.0001,0.001,0.01,0.1,1,5,10,20,40,80,160]
wyniki_elastic = pd.DataFrame()

for x in alpha_values_elastic:
    for y in l1_ratio_values:
        elasticdf = elastic_net(x, y) 
        tn, fp, fn, tp = confusion_matrix(ytest,elastic_net(x,y) > 0.5).ravel()
        accuracy =((tp + tn)/(tn + fp + fn + tp))
        dic = {'alpha':x , 'l1':y, 'accuracy':accuracy}
        wyniki_elastic = wyniki_elastic.append(dic, ignore_index = True)

print(wyniki_elastic)
print('Najwyzsza trafnosci:',wyniki_elastic['accuracy'].max())
print('Wartosc alpha dla takiego wyniku',wyniki_elastic['alpha'][wyniki_elastic['accuracy'].idxmax()])
print('Wartosc l1 dla takiego wyniku',wyniki_elastic['l1'][wyniki_elastic['accuracy'].idxmax()])

      accuracy      alpha    l1
0     0.869565    0.00001  0.01
1     0.869565    0.00001  0.02
2     0.869565    0.00001  0.03
3     0.869565    0.00001  0.04
4     0.869565    0.00001  0.05
...        ...        ...   ...
1183  0.565217  160.00000  0.95
1184  0.565217  160.00000  0.96
1185  0.565217  160.00000  0.97
1186  0.565217  160.00000  0.98
1187  0.565217  160.00000  0.99

[1188 rows x 3 columns]
Najwyzsza trafnosci: 0.8840579710144928
Wartosc alpha dla takiego wyniku 0.1
Wartosc l1 dla takiego wyniku 0.060000000000000005
